In [10]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func

In [11]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='ZAF') | Q(sovereign__icontains='ZAF')
)
mpaset.values('country').distinct()

<QuerySet [{'country': 'ZAF'}, {'country': 'ZAF/MOZ'}]>

In [12]:
mpas_all_nogeom.filter(wdpa_id=0).update(wdpa_id=None)
mpas_all_nogeom.filter(wdpa_id=0).count()

0

In [13]:
count = 0
for m in mpaset:
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > datetime.datetime(2014, 11, 5):
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2018-10-24 09:58:47.257651 count: 8 mpa_id: 67704044 wdpa_id: 306178
2 2018-10-24 10:09:28.440747 count: 5 mpa_id: 68808364 wdpa_id: 555512160
3 2018-10-24 10:09:31.863068 count: 15 mpa_id: 7705440 wdpa_id: 555512160
4 2018-10-24 10:09:35.332193 count: 6 mpa_id: 68808363 wdpa_id: 555512160
5 2016-02-10 15:08:33.838697 count: 3 mpa_id: 4838 wdpa_id: 315489
6 2017-08-08 13:34:48.709163 count: 2 mpa_id: 67704828 wdpa_id: 315479
7 2015-04-07 11:47:04.947382 count: 2 mpa_id: 2334 wdpa_id: 68170
8 2015-03-26 10:46:56.236162 count: 2 mpa_id: 12354 wdpa_id: 903059
9 2016-02-10 09:26:15.779302 count: 3 mpa_id: 4841 wdpa_id: 315492
10 2015-05-06 11:04:27.908079 count: 1 mpa_id: 60000131 wdpa_id: 881
11 2016-12-06 14:53:39.960924 count: 2 mpa_id: 68804907 wdpa_id: 351699
12 2017-02-21 09:38:23.451843 count: 3 mpa_id: 67703502 wdpa_id: 198302
13 2014-11-05 16:21:25.924720 count: 1 mpa_id: 9748 wdpa_id: 17368
14 2016-02-10 11:13:04.719559 count: 3 mpa_id: 15138 wdpa_id: 555563467
15 2016-02-10 09

In [14]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [15]:
remove_log = '/home/mpatlas/workspace/log/removewdpa_ZAF_log.json'
wdpa2remove = merge.getRemoveWdpaList(logfile=remove_log)

In [16]:
len(wdpa2remove)

7

In [17]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

7

In [18]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

68804905 351697 ZAF ZAF True Not Reported None Namaqualand Marine Protected Area
68804901 351693 ZAF ZAF True All None Cape of Good Hope Restricted Zone Marine Protected Area
68804907 351699 ZAF ZAF True All None Paulsberg Restricted Zone Marine Protected Area
68804908 351700 ZAF ZAF True All None St James Restricted Zone Marine Protected Area
68804900 351692 ZAF ZAF True All None Boulders Restricted Zone Marine Protected Area
9695 875 ZAF ZAF False Not Reported 0.0 Addo-Elephant National Park
68804903 351695 ZAF ZAF True All None Karbonkelberg Restricted Zone Marine Protected Area


In [19]:
merge.removeMpasByWdpaId(wdpa2remove)

In [20]:
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [21]:
wdpa2add = merge.getAddWdpaList()

In [22]:
len(wdpa2add)

0

In [23]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

In [24]:
len(wdpapid2add)

0

In [16]:
# Assign records with zones to be the first zone.  Do not rerun.
# In separate notebook

In [25]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)

0

In [26]:
reload(merge)
addpoint_geolog = '/home/mpatlas/workspace/log/addpoint_ZAF_log.json'
addpoly_log = '/home/mpatlas/workspace/log/addpoly_ZAF_log.json'
addpoly_geolog = '/home/mpatlas/workspace/log/addpoly_ZAF_geo_log.json'

In [27]:
points = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=None, geologfile=addpoint_geolog, dryrun=True)

True

In [28]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, geologfile=addpoly_geolog, dryrun=True)
# post_save.connect(mpa_post_save, sender=Mpa)

True

In [30]:
reload(merge)
updatepoint_geolog = '/home/mpatlas/workspace/log/updatepoint_ZAF_log.json'
updatepoly_log = '/home/mpatlas/workspace/log/updatepoly_ZAF_log.json'
updatepoly_geolog = '/home/mpatlas/workspace/log/updatepoly_ZAF_geo_log.json'

In [31]:
wdpa_filter = (
    Q(iso3__icontains='ZAF') | Q(parent_iso3__icontains='ZAF')
)
polys = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))



In [32]:
len(allpid2update)

26

In [33]:
updatepoints = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=None, geologfile=updatepoint_geolog, dryrun=True)

True

In [34]:
updatepolys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update)
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, geologfile=updatepoly_geolog, dryrun=True)

1/41 adding/updating wdpa_pid 315477 with mpa_id 4827
    PID: 315477 has 5 field diffs  and GEOM diff
2/41 adding/updating wdpa_pid 315477 with mpa_id 67704827
    PID: 315477 has 9 field diffs  and GEOM diff
3/41 adding/updating wdpa_pid 315480 with mpa_id 4829
    PID: 315480 has 7 field diffs  and GEOM diff
4/41 adding/updating wdpa_pid 315481 with mpa_id 4830
    PID: 315481 has 5 field diffs  and GEOM diff
5/41 adding/updating wdpa_pid 315481 with mpa_id 67704830
    PID: 315481 has 10 field diffs  and GEOM diff
6/41 adding/updating wdpa_pid 315490 with mpa_id 4839
    PID: 315490 has 6 field diffs  and GEOM diff
7/41 adding/updating wdpa_pid 300543 with mpa_id 3640
    PID: 300543 has 6 field diffs  and GEOM diff
8/41 adding/updating wdpa_pid 300546 with mpa_id 3641
    PID: 300546 has 10 field diffs  and GEOM diff
9/41 adding/updating wdpa_pid 300546 with mpa_id 67703641
    PID: 300546 has 11 field diffs  and GEOM diff
10/41 adding/updating wdpa_pid 306178 with mpa_id 67704044

True

In [35]:
mpaset.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

66

In [36]:
nt = ('All', 'Part', 'None')
ntupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

11

In [37]:
ntareaupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

1

In [38]:
ntchanges = ntupdate | ntareaupdate
for n in ntchanges:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

3641 300546 300546 Dwesa-Cwebe Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 0.0
67703641 300546 300546 Dwesa-Cwebe Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 None
67704044 306178 306178 Table Mountain National Park Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 56.4
67704047 306181 306181 Pondoland Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 None
67704832 315483 315483 Langebaan Lagoon Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 None
67704827 315477 315477 Betty's Bay Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 None
4829 315480 315480 Goukamma Marine Protected Area Marine Protected Area Marine Protected Area Not Reported Part 0.0 0.0
67704045 306179 306179 Aliwal Shoal Marine Protected Area Marine Protected Area Marine Protected 

In [39]:
ntchanges = ntupdate | ntareaupdate

import csv

with open('/home/mpatlas/workspace/log/notakechange_ZAF_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['mpa_id', 'wdpa_id', 'wdpa_pid', 'name', 'designation', 'designation_eng', 'no_take_wdpa', 'no_take_mpatlas', 'no_take_area_wdpa', 'no_take_area_mpatlas']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in ntchanges.values(*fieldnames):
        writer.writerow(row)

In [40]:
for n in ntchanges:
    try:
        w2018 = Wdpa2018Poly.objects.get(wdpa_pid=n.wdpa_pid)
    except:
        w2018 = Wdpa2018Point.objects.get(wdpa_pid=n.wdpa_pid)
    try:
        w2014 = Wdpa2014Polygon.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
    except:
        try:
            w2014 = Wdpa2014Point.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
        except:
            try:
                w2014 = Wdpa2014Polygon.objects.filter(wdpa_id=F('wdpa_id'))[0]
            except:
                w2014 = Wdpa2014Point.objects.filter(wdpa_id=F('wdpa_id'))[0]
    if (n.no_take == w2014.no_take) or (n.no_take_area == w2014.no_tk_area):
        print(w2018.name, w2018.desig, n.no_take_mpatlas, w2018.no_take, w2014.no_take, n.no_take_area_mpatlas, w2018.no_tk_area, w2014.no_tk_area)

Dwesa-Cwebe Marine Protected Area Marine Protected Area Part Not Reported Not Applicable 0.0 0.0 0.0
Dwesa-Cwebe Marine Protected Area Marine Protected Area Part Not Reported Not Applicable None 0.0 0.0
Pondoland Marine Protected Area Marine Protected Area Part Not Reported Not Applicable None 0.0 0.0
Langebaan Lagoon Marine Protected Area Marine Protected Area Part Not Reported Not Applicable None 0.0 0.0
Betty's Bay Marine Protected Area Marine Protected Area Part Not Reported Not Applicable None 0.0 0.0
Goukamma Marine Protected Area Marine Protected Area Part Not Reported Not Applicable 0.0 0.0 0.0
Aliwal Shoal Marine Protected Area Marine Protected Area Part Not Reported Not Applicable None 0.0 0.0
Bird Island Group Marine Protected Area Marine Protected Area All Not Reported Not Applicable None 0.0 0.0
De Hoop Marine Protected Area Marine Protected Area All Not Reported Not Applicable None 0.0 0.0
Helderberg Marine Protected Area Marine Protected Area All Not Reported Not Applica

In [42]:
ntchanges2 = ntupdate | ntareaupdate
ntchanges.count()

11

In [43]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='ZAF') | Q(sovereign__icontains='ZAF')
)

from mpatlas.utils import cartompa
cartompa.purgeCartoMpas()
cartompa.updateAllMpas(mpas=mpaset)

Processing 10 of 100 mpa records at a time
Records [0 - 9]
ERROR processing mpa 9106:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9106
ERROR processing mpa 9083:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9083
Records [10 - 19]
ERROR processing mpa 9109:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9109
Records [20 - 29]
ERROR processing mpa 9117:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9117
ERROR processing mpa 9103:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9103
ERROR processing mpa 68807928:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807928
ERROR processing mpa 9118:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 9118
Records [30 - 39]
Records [40 - 49]
Records [50 - 59]
Records [60 - 69]
Records [70 - 79]
Records [80 - 89]
Records [90 - 99]
TOTAL 17.572350025177002 sec elapsed


[9106, 9083, 9109, 9117, 9103, 68807928, 9118]